In [1]:
import $ivy.`org.typelevel::cats-core:2.9.0`

import cats._
import cats.syntax.all._
import cats.data.Nested

extension [A](a: A) {
  inline def |>[B](inline f: A => B): B = f(a)
}

import $ivy.$                               


import cats._

import cats.syntax.all._

import cats.data.Nested


defined extension methods 

In [2]:
// "Schema"
class V()
class E(val src: V, val tgt: V)
class G(val vs: List[V], val es: List[E])

defined class V
defined class E
defined class G

In [3]:
// "Flat"
object QF:
    // Edge "Attrs"
    def src(e: E): V = e.src 
    def tgt(e: E): V = e.tgt

    // Graph "Attrs"
    def vs(g: G): List[V] = g.vs 
    def es(g: G): List[E] = g.es

    // Bundles 
    def outgoing(v: V)(using g: G): List[E] = g.es.filter(src(_) == v)
    def incoming(v: V)(using g: G): List[E] = g.es.filter(tgt(_) == v)

    // Spans
    def oneighbors(v: V)(using g: G): List[V] = g.vs.filter(outgoing(v).map(tgt).contains(_))
    def ineighbors(v: V)(using g: G): List[V] = g.vs.filter(incoming(v).map(src).contains(_))
    def neighbors(v: V)(using g: G): List[V] = oneighbors(v) ++ ineighbors(v)

    // Generic
    def sum[A](using num: Numeric[A])(as: List[A]): A = as.sum
    def size[A](as: List[A]): Int = as.size
    def range(i: Int): List[Int] = List.range(0, i)

defined object QF

In [4]:
// "Nested"
object QN:
    // Edge "Attrs"
    def src[F[_]](fe: F[E])(using F: Functor[F]): F[V] = fe map QF.src
    def tgt[F[_]](fe: F[E])(using F: Functor[F]): F[V] = fe map QF.tgt

    // Graph "Attrs"
    def vs[F[_]](fg: F[G])(using F: Functor[F]): Nested[F, List, V] = Nested(fg map QF.vs)
    def es[F[_]](fg: F[G])(using F: Functor[F]): Nested[F, List, E] = Nested(fg map QF.es)

    // Bundles
    def outgoing[F[_]](fv: F[V])(using g: G)(using F: Functor[F]): Nested[F, List, E] = Nested(fv map QF.outgoing)
    def incoming[F[_]](fv: F[V])(using g: G)(using F: Functor[F]): Nested[F, List, E] = Nested(fv map QF.incoming)

    // Spans
    def oneighbors[F[_]](fv: F[V])(using g: G)(using F: Functor[F]): Nested[F, List, V] = Nested(fv map QF.oneighbors)
    def ineighbors[F[_]](fv: F[V])(using g: G)(using F: Functor[F]): Nested[F, List, V] = Nested(fv map QF.ineighbors)
    def neighbors[F[_]](fv: F[V])(using g: G)(using F: Functor[F]): Nested[F, List, V] = Nested(fv map QF.neighbors)

    // Generic
    def sum[A, F[_]](fas: Nested[F, List, A])(using F: Functor[F])(using num: Numeric[A]): F[A] = fas.value map (QF.sum)
    def size[A, F[_]](fas: Nested[F, List, A])(using F: Functor[F]): F[Int] = fas.value map (QF.size)
    def range[F[_]](fas: F[Int])(using F: Functor[F]): Nested[F, List, Int] = Nested(fas map (QF.range))

defined object QN

In [5]:
// "Instance"

val v0 = V()
val v1 = V()
val v2 = V()
val e0 = E(v0, v1)
val e1 = E(v1, v2)

val g = G(List(v0,v1,v2), List(e0,e1))

given global: G = g

v0: V = ammonite.$sess.cmd1$Helper$V@3b747aa5
v1: V = ammonite.$sess.cmd1$Helper$V@68d4bc89
v2: V = ammonite.$sess.cmd1$Helper$V@2a080882
e0: E = ammonite.$sess.cmd1$Helper$E@39c7758c
e1: E = ammonite.$sess.cmd1$Helper$E@6faa4aea
g: G = ammonite.$sess.cmd1$Helper$G@4c23ee26
global: G = <given>

In [6]:
e0.toString

res5: String = "ammonite.$sess.cmd1$Helper$E@39c7758c"

In [7]:
val fg = List(g)

fg: List[G] = List(ammonite.$sess.cmd1$Helper$G@4c23ee26)

In [8]:
fg 
    |> QN.vs         // Produce "List"

res7: Nested[List, List, V] = Nested(
  value = List(
    List(
      ammonite.$sess.cmd1$Helper$V@3b747aa5,
      ammonite.$sess.cmd1$Helper$V@68d4bc89,
      ammonite.$sess.cmd1$Helper$V@2a080882
    )
  )
)

In [9]:
fg 
    |> QN.vs         // Produce "List"
    |> QN.neighbors  // Produce "List"

res8: Nested[_root_.cats.data.Nested[[A >: scala.Nothing <: scala.Any] => _root_.cats.data.Nested[scala.collection.immutable.List, scala.List, A], scala.List, ammonite.$sess.cmd3.wrapper.cmd1.V], List, V] = Nested(
  value = Nested(
    value = List(
      List(
        List(ammonite.$sess.cmd1$Helper$V@68d4bc89),
        List(
          ammonite.$sess.cmd1$Helper$V@2a080882,
          ammonite.$sess.cmd1$Helper$V@3b747aa5
        ),
        List(ammonite.$sess.cmd1$Helper$V@68d4bc89)
      )
    )
  )
)

In [10]:
fg 
    |> QN.vs         // Produce "List"
    |> QN.neighbors  // Produce "List"
    |> QN.size       // Reduce "List"

res9: Nested[List, List, Int] = Nested(value = List(List(1, 2, 1)))

In [11]:
fg 
    |> QN.vs         // Produce "List"
    |> QN.neighbors  // Produce "List"
    |> QN.size       // Reduce "List"
    |> QN.range      // Produce "List"
    |> QN.sum        // Reduce "List"

res10: Nested[List, List, Int] = Nested(value = List(List(0, 1, 0)))